In [1]:
### going to import household data and train a classifier on the data that takes as input the datetime information and outputs a daily load profile

import pandas as pd
import numpy as np
import os
os.chdir('..')
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, f1_score, accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
import holidays
import datetime

from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from utils import *





In [2]:
df = pd.read_csv(r'data\cleaned_data\5_household\power\SFH36_demand_W.csv', index_col=0, parse_dates=True)

scaler = MinMaxScaler()

df[df.columns] = scaler.fit_transform(df[df.columns])

df = timeseries_dataframe_pivot(df).dropna()


In [80]:
# past information
def calc_rolling_sum_of_load(df, n_days):
    df['rolling_sum'] = df.sum(axis=1).rolling(n_days).sum().shift(1)
    df = df.dropna()
    return df

df = calc_rolling_sum_of_load(df, 7)
df["load_of_last_day"] = df.iloc[:,:-1].sum(axis=1).shift(1).dropna(inplace=True)
df.head()


,00:00:00,00:15:00,00:30:00,00:45:00,01:00:00,01:15:00,01:30:00,01:45:00,02:00:00,02:15:00,...,21:45:00,22:00:00,22:15:00,22:30:00,22:45:00,23:00:00,23:15:00,23:30:00,23:45:00,rolling_sum
date,,,,,,,,,,,,,,,,,,,,,
2018-05-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.554101e+02
2018-05-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.773497e+02
2018-05-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.953993e-13
2018-05-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00
2018-05-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00


In [3]:
years = get_year_list(df)
df_holidays_dummies = get_holidays(years, "DE")
df_holidays = create_holiday_features(df, df_holidays_dummies)

NameError: name 'holidays' is not defined

In [64]:
df = create_datetime_features(df)
df = pd.concat([df, df_holidays], axis=1)
df = df.astype('float32')

In [60]:
# split the data into train and test

train, test = train_test_split(df, test_size=0.2, shuffle=False)

train = train.values
test = test.values

# scale the data

scaler = MinMaxScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

#features and target

X_train = train[:,96:]
y_train = train[:,:96]

X_test = test[:,96:]
y_test = test[:,:96]

# define the model
model = XGBRegressor(
    objective='reg:pseudohubererror'
    )



c:\Users\nik\miniconda3\envs\gpu2\lib\site-packages\sklearn\preprocessing\_data.py:473: RuntimeWarning:

All-NaN slice encountered

c:\Users\nik\miniconda3\envs\gpu2\lib\site-packages\sklearn\preprocessing\_data.py:474: RuntimeWarning:

All-NaN slice encountered



In [48]:

param_distributions = {'n_estimators': np.arange(100, 1000, 100),
                       'objective': ['reg:pseudohubererror', 'reg:squarederror','reg:tweedie'],
                       'max_depth': [9, 12],
                       'learning_rate': [0.001, 0.01,0.1],
                       'subsample': [0.5, 0.7, 0.9],
                       'colsample_bytree': [0.5, 0.7, 0.9],
                       'reg_alpha': [0, 0.1, 1, 10],
                       'reg_lambda': [0, 0.1, 1, 10]}

# define the search
search = RandomizedSearchCV(model, param_distributions, n_iter=100, scoring='neg_root_mean_squared_error', n_jobs=-1, cv=3, random_state=42)

In [ ]:
search.fit(X_train, y_train)
model = search.best_estimator_

In [49]:
model.fit(X_train, y_train)



XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             objective='reg:pseudohubererror', predictor=None, ...)

In [50]:
predictions = model.predict(X_test).reshape(-1,1).flatten()
predictions[predictions < 0] = 0

# create benchmark that just shifts the data by 24 hours and pads the beginning with zeros

y_test_reshaped =  y_test.reshape(-1,1).flatten()

gt = pd.Series(y_test_reshaped)
benchmark_day = gt.shift(96).fillna(0)
benchmark_week = gt.shift(96*7).fillna(0)

In [51]:
df_compare = pd.DataFrame({'gt':gt, 
                            'predictions': predictions,
                            'benchmark_day': benchmark_day,
                            'benchmark_week': benchmark_week}
                            )[96*7:]

px.line(df_compare,  title='XGBoost Regression')

In [52]:
# calculate error metrics, between the first column and all other columns

def calc_metrics(df_compare, metrics):
    "calculates metrics for a dataframe with a ground truth column and predictions, ground truth column must be the first column"
    metric_series_list = {}
    for metric in metrics:
        metric_name = metric.__name__
        metric_result = df_compare.apply(lambda x: metric(x, df_compare.iloc[:,0]), axis=0)
        if metric.__name__ == 'mean_squared_error':
            metric_result = np.sqrt(metric_result)
            metric_name = 'root_mean_squared_error'

        metric_series_list[metric_name] = metric_result

    df_metrics = pd.DataFrame(metric_series_list)
    return df_metrics

calc_metrics(df_compare[7*96:], [mean_absolute_error, mean_squared_error, r2_score])



,mean_absolute_error,root_mean_squared_error,r2_score
gt,0.000000,0.000000,1.000000
predictions,0.138447,0.198894,-0.304762
benchmark_day,0.101334,0.183422,-0.501848
benchmark_week,0.098700,0.179539,-0.465283


In [106]:
benchmark_day_per_day = benchmark_day.values.reshape(predictions.shape)[:,7:]
benchmark_week_per_day = benchmark_week.values.reshape(predictions.shape)[:,7:]

### Visualization

In [143]:
df_daily_sum = pd.DataFrame({'pred_sums': predictions.sum(axis=0)[7:], 
                            'gt_sums': y_test.sum(axis=0)[7:],
                            'benchmark_day_sums': benchmark_day_per_day.sum(axis=0),
                            'benchmark_week_sums': benchmark_week_per_day.sum(axis=0)
                             })[7:]

px.line(df_daily_sum, title='XGBoost Regression')